# కార్ట్పోల్ బ్యాలెన్సింగ్ కోసం RL శిక్షణ

ఈ నోట్‌బుక్ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) భాగం. ఇది [అధికారిక PyTorch ట్యుటోరియల్](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) మరియు [ఈ కార్ట్పోల్ PyTorch అమలు](https://github.com/yc930401/Actor-Critic-pytorch) నుండి ప్రేరణ పొందింది.

ఈ ఉదాహరణలో, మేము RL ఉపయోగించి ఒక మోడల్‌ను శిక్షణ ఇస్తాము, ఇది ఒక కార్ట్‌పై ఉన్న కంబళిని సమతుల్యం చేయగలదు, ఆ కార్ట్ ఎడమ మరియు కుడి వైపుకు ఆడవచ్చు. కంబళిని సిమ్యులేట్ చేయడానికి మేము [OpenAI Gym](https://www.gymlibrary.ml/) వాతావరణాన్ని ఉపయోగిస్తాము.

> **గమనిక**: మీరు ఈ పాఠం కోడ్‌ను స్థానికంగా (ఉదా: Visual Studio Code నుండి) నడపవచ్చు, అప్పుడు సిమ్యులేషన్ కొత్త విండోలో తెరుచుకుంటుంది. ఆన్‌లైన్‌లో కోడ్ నడిపేటప్పుడు, కొంత మార్పులు చేయాల్సి రావచ్చు, వివరాలు [ఇక్కడ](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) ఉన్నాయి.

ముందుగా Gym ఇన్‌స్టాల్ అయిందో లేదో చూసుకుందాం:


In [ ]:
import sys
!{sys.executable} -m pip install gym

ఇప్పుడు మనం CartPole వాతావరణాన్ని సృష్టించి దానిపై ఎలా ఆపరేట్ చేయాలో చూద్దాం. ఒక వాతావరణానికి క్రింది లక్షణాలు ఉంటాయి:

* **Action space** అనేది సిమ్యులేషన్ ప్రతి దశలో మనం చేయగలిగే సాధ్యమైన చర్యల సమాహారం
* **Observation space** అనేది మనం చేయగలిగే పరిశీలనల స్థలం


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

సిమ్యులేషన్ ఎలా పనిచేస్తుందో చూద్దాం. క్రింది లూప్ సిమ్యులేషన్‌ను నడిపిస్తుంది, `env.step` ముగింపు ఫ్లాగ్ `done` ను తిరిగి ఇవ్వకపోవడం వరకు. మనం యాదృచ్ఛికంగా చర్యలను ఎంచుకుంటాము `env.action_space.sample()` ఉపయోగించి, అంటే ప్రయోగం చాలా త్వరగా విఫలమవుతుంది (CartPole వాతావరణం CartPole వేగం, దాని స్థానం లేదా కోణం కొన్ని పరిమితుల వెలుపల ఉన్నప్పుడు ముగుస్తుంది).

> సిమ్యులేషన్ కొత్త విండోలో తెరుస్తుంది. మీరు కోడ్‌ను అనేక సార్లు నడిపించి దాని ప్రవర్తనను చూడవచ్చు.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

మీరు గమనించవచ్చు, ఆబ్జర్వేషన్లు 4 సంఖ్యలను కలిగి ఉంటాయి. అవి:
- కార్ట్ యొక్క స్థానం
- కార్ట్ యొక్క వేగం
- ధ్రువం యొక్క కోణం
- ధ్రువం యొక్క తిరుగుడు వేగం

`rew` అనేది ప్రతి దశలో మేము పొందే రివార్డు. CartPole పరిసరంలో మీరు ప్రతి సిమ్యులేషన్ దశకు 1 పాయింట్ రివార్డు పొందుతారని మీరు చూడవచ్చు, మరియు లక్ష్యం మొత్తం రివార్డును గరిష్టం చేయడం, అంటే CartPole పడకుండా సమతుల్యం నిలబెట్టగలిగే సమయం.

పునరుద్ధరణ అభ్యాస సమయంలో, మా లక్ష్యం ఒక **పాలసీ** $\pi$ ను శిక్షణ ఇవ్వడం, ఇది ప్రతి స్థితి $s$ కోసం ఏ చర్య $a$ తీసుకోవాలో చెప్పుతుంది, అంటే $a = \pi(s)$.

మీరు ప్రాబబిలిస్టిక్ పరిష్కారాన్ని కోరుకుంటే, పాలసీని ప్రతి చర్యకు సంబంధించిన ప్రాబబిలిటీల సమితిని ఇచ్చే విధంగా భావించవచ్చు, అంటే $\pi(a|s)$ అనగా స్థితి $s$ వద్ద చర్య $a$ తీసుకోవాల్సిన ప్రాబబిలిటీ.

## పాలసీ గ్రేడియంట్ పద్ధతి

సరళమైన RL అల్గోరిథంలో, దీనిని **పాలసీ గ్రేడియంట్** అంటారు, మేము తదుపరి చర్యను అంచనా వేయడానికి ఒక న్యూరల్ నెట్‌వర్క్‌ను శిక్షణ ఇస్తాము.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

మేము నెట్‌వర్క్‌ను అనేక ప్రయోగాలు నిర్వహించి శిక్షణ ఇస్తాము, మరియు ప్రతి రన్ తర్వాత మా నెట్‌వర్క్‌ను నవీకరిస్తాము. ప్రయోగాన్ని నిర్వహించి ఫలితాలను (అంటే **ట్రేస్**) తిరిగి ఇచ్చే ఫంక్షన్‌ను నిర్వచిద్దాం - అన్ని స్థితులు, చర్యలు (మరియు వాటి సిఫారసు చేసిన సంభావ్యతలు), మరియు బహుమతులు:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

మీరు ఒక ఎపిసోడ్‌ను శిక్షణ పొందని నెట్‌వర్క్‌తో నడిపించి, మొత్తం బహుమతి (అంటే ఎపిసోడ్ పొడవు) చాలా తక్కువగా ఉన్నదని గమనించవచ్చు:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

పాలసీ గ్రాడియెంట్ అల్గోరిథం యొక్క ఒక క్లిష్టమైన అంశం **డిస్కౌంటెడ్ రివార్డ్స్** ఉపయోగించడం. ఆలోచన ఏమిటంటే, ఆట యొక్క ప్రతి దశలో మొత్తం రివార్డ్స్ వెక్టర్‌ను గణించటం, మరియు ఈ ప్రక్రియలో మేము ప్రారంభ రివార్డ్స్‌ను కొన్ని గుణకం $gamma$ ఉపయోగించి డిస్కౌంట్ చేస్తాము. మేము ఫలితంగా వచ్చిన వెక్టర్‌ను సాధారణీకరించ also, ఎందుకంటే దీన్ని మా శిక్షణను ప్రభావితం చేయడానికి బరువుగా ఉపయోగిస్తాము:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

ఇప్పుడు నిజమైన శిక్షణ ప్రారంభిద్దాం! మనం 300 ఎపిసోడ్లు నడిపించబోతున్నాము, ప్రతి ఎపిసోడ్‌లో మనం ఈ క్రింది పనులు చేస్తాము:

1. ప్రయోగాన్ని నడిపించి ట్రేస్ సేకరించండి  
2. తీసుకున్న చర్యలు మరియు అంచనా probabilities మధ్య తేడా (`gradients`) లెక్కించండి. తేడా తక్కువగా ఉంటే, మనం సరైన చర్య తీసుకున్నామని ఎక్కువ నమ్మకం ఉంటుంది.  
3. డిస్కౌంటెడ్ రివార్డ్స్ లెక్కించి gradients ను వాటితో గుణించండి - ఇది ఎక్కువ రివార్డ్ ఉన్న దశలు తక్కువ రివార్డ్ ఉన్న దశల కంటే ఫలితంపై ఎక్కువ ప్రభావం చూపుతాయని నిర్ధారిస్తుంది  
4. మన న్యూరల్ నెట్‌వర్క్ కోసం ఆశించిన లక్ష్య చర్యలు, రన్ సమయంలో అంచనా probabilities నుండి కొంత భాగం మరియు లెక్కించిన gradients నుండి కొంత భాగం తీసుకుంటాయి. gradients మరియు రివార్డ్స్‌ను ఎంత మేరకు పరిగణలోకి తీసుకోవాలో నిర్ణయించడానికి `alpha` పారామీటర్ ఉపయోగిస్తాము - దీనిని రీఫోర్స్‌మెంట్ అల్గోరిథం యొక్క *లెర్నింగ్ రేట్* అంటారు.  
5. చివరగా, మనం స్టేట్స్ మరియు ఆశించిన చర్యలపై నెట్‌వర్క్‌ను శిక్షణ ఇస్తాము, మరియు ఈ ప్రక్రియను పునరావృతం చేస్తాము.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

ఇప్పుడు ఫలితాన్ని చూడటానికి రేండరింగ్‌తో ఎపిసోడ్‌ను నడపుదాం:


In [ ]:
_ = run_episode(render=True)

ఆశిస్తున్నాను, మీరు ఇప్పుడు పొల్ బాగా సంతులనం అవుతుందని చూడగలుగుతారు!

## యాక్టర్-క్రిటిక్ మోడల్

యాక్టర్-క్రిటిక్ మోడల్ అనేది పాలసీ గ్రాడియెంట్ల మరింత అభివృద్ధి, ఇందులో మనం పాలసీ మరియు అంచనా వేయబడిన రివార్డులను నేర్చుకునే న్యూరల్ నెట్‌వర్క్‌ను నిర్మిస్తాము. నెట్‌వర్క్‌కు రెండు అవుట్‌పుట్లు ఉంటాయి (లేదా మీరు దీన్ని రెండు వేర్వేరు నెట్‌వర్క్‌లుగా చూడవచ్చు):
* **యాక్టర్** పాలసీ గ్రాడియెంట్ మోడల్‌లో ఉన్నట్లుగా, మనకు స్టేట్ ప్రాబబిలిటీ పంపిణీని ఇచ్చి తీసుకోవాల్సిన చర్యను సిఫార్సు చేస్తుంది
* **క్రిటిక్** ఆ చర్యల నుండి రివార్డ్ ఎంత ఉంటుందో అంచనా వేస్తుంది. ఇది ఇచ్చిన స్టేట్‌లో భవిష్యత్తులో మొత్తం అంచనా వేయబడిన రివార్డులను తిరిగి ఇస్తుంది.

ఇలాంటి మోడల్‌ను నిర్వచిద్దాం:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

మనం మా `discounted_rewards` మరియు `run_episode` ఫంక్షన్లను కొంచెం మార్చుకోవాలి:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


ఇప్పుడు మనం ప్రధాన శిక్షణ లూప్‌ను నడిపించబోతున్నాము. సరైన నష్ట ఫంక్షన్లను లెక్కించి మరియు నెట్‌వర్క్ పారామితులను నవీకరించడం ద్వారా మాన్యువల్ నెట్‌వర్క్ శిక్షణ ప్రక్రియను ఉపయోగిస్తాము:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

చివరికి, మనం వాతావరణాన్ని మూసివేయండి.


In [ ]:
env.close()

## ముఖ్యాంశాలు

ఈ డెమోలో మనం రెండు RL అల్గోరిథమ్స్ చూశాము: సింపుల్ పాలసీ గ్రేడియంట్ మరియు మరింత సున్నితమైన యాక్టర్-క్రిటిక్. ఆ అల్గోరిథమ్స్ స్టేట్, యాక్షన్ మరియు రివార్డ్ అనే సారాంశ భావాలతో పనిచేస్తాయని మీరు చూడవచ్చు - అందువల్ల అవి చాలా భిన్నమైన వాతావరణాల్లో కూడా వర్తించవచ్చు.

రీఫోర్స్‌మెంట్ లెర్నింగ్ మనకు సమస్యను పరిష్కరించడానికి ఉత్తమ వ్యూహాన్ని చివరి రివార్డ్‌ను చూసి నేర్చుకునే అవకాశం ఇస్తుంది. లేబుల్ చేయబడిన డేటాసెట్‌ల అవసరం లేకపోవడం వల్ల మనం సిమ్యులేషన్లను ఎన్నో సార్లు పునరావృతం చేసి మన మోడల్స్‌ను మెరుగుపరచుకోవచ్చు. అయితే, RLలో ఇంకా అనేక సవాళ్లు ఉన్నాయి, మీరు ఈ ఆసక్తికరమైన AI రంగంపై మరింత దృష్టి పెట్టాలనుకుంటే అవి తెలుసుకోవచ్చు.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
